In [1]:
from stable_baselines3 import PPO
import torch
import numpy as np

## Warning:
This "normalization" replaces whatever you put in with 100.0
This yields a safe network. It just doesn't yield a network you'd actually want to use...

In [45]:
class OnnxableActionPolicy(torch.nn.Module):
    def __init__(self, extractor, action_net, value_net):
        super(OnnxableActionPolicy, self).__init__()
        self.extractor = extractor
        self.action_net = action_net
        self.value_net = value_net
        normalize_linear1 = torch.nn.Linear(1, 4)
        # 100* ((max(0,x) - max(0,-x)) - max(0,x-1) + max(0,-x-1))
        normalize_linear1.weight.data = torch.Tensor([[1],[-1],[1],[-1]])
        normalize_linear1.bias.data=torch.Tensor([0,0,-1,-1])
        #print(normalize_linear1.weight)
        #print(normalize_linear1.bias)
        A = 100-1e-6
        normalize_linear2 = torch.nn.Linear(3,1)
        normalize_linear2.weight.data = torch.Tensor([[0,0,0,0]])#A,-A,-A,A]])
        normalize_linear2.bias.data=torch.Tensor([100.0])
        self.normalizer = torch.nn.Sequential(
            normalize_linear1,
            torch.nn.ReLU(),
            normalize_linear2)

    def forward(self, observation):
        # NOTE: You may have to process (normalize) observation in the correct
        #       way before using this. See `common.preprocessing.preprocess_obs`
        action_hidden, value_hidden = self.extractor(observation)
        action = self.action_net(action_hidden)
        return self.normalizer(action) #, self.value_net(value_hidden)

In [46]:
# Example: model = PPO("MlpPolicy", "Pendulum-v0")
model = PPO.load("model_backup/ppo_acc_bigger_400000_steps_new")
model.policy.to("cpu")
onnxable_model = OnnxableActionPolicy(model.policy.mlp_extractor, model.policy.action_net, model.policy.value_net)

In [47]:
onnxable_model.normalizer(torch.Tensor([2]))

tensor([100.], grad_fn=<AddBackward0>)

In [48]:
dummy_input = torch.randn(1, 2)
torch.onnx.export(onnxable_model, dummy_input, "ppo_acc_bigger2_200000_steps.onnx", opset_version=9)

In [49]:
##### Load and test with onnx

import onnx
import onnxruntime as ort
import numpy as np

In [50]:
onnx_model = onnx.load("ppo_acc_bigger2_200000_steps.onnx")
onnx.checker.check_model(onnx_model)

In [51]:
onnx_model = onnx.load("ppo_acc_bigger2_200000_steps.onnx")
onnx.checker.check_model(onnx_model)

observation = np.zeros((1, 2)).astype(np.float32)
ort_sess = ort.InferenceSession("ppo_acc_bigger2_200000_steps.onnx")

In [52]:
print(ort_sess.run(None, {'input.1': [[50,-99.0]]}))

[array([[100.]], dtype=float32)]


In [55]:
import gym
import acc2

In [56]:
env = gym.make("acc-variant-v3")

/home/steuber/anaconda3/envs/nnequiv-tf1/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Environment '<class 'acc2.ACCEnv2'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [62]:
obs = env.reset()
obs = [50,-99.0]
env.unwrapped.state = obs
for i in range(0,300):
    action = ort_sess.run(None, {'input.1': [[obs[0],obs[1]]]})[0][0]/100
    action = np.clip(action,-100.,100.)
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        print("DONE")
        print(obs)
        env.reset()

DONE
[203. 201.]
DONE
[ -2.56749 -45.21249]
DONE
[208.70324 203.9337 ]
DONE
[205.1283  155.79213]
DONE
[203.55918 201.60927]
DONE
[200.80912 149.63452]
DONE
[207.04463 160.11322]
DONE
[212.01846 208.24332]
DONE
[215.48387 197.1811 ]
DONE
[210.51501 199.10815]
DONE
[ -0.60513 -46.84688]
DONE
[219.5748  224.45196]
DONE
[206.81571 199.54731]
DONE
[214.56567 208.9313 ]
DONE
[201.10369 200.55109]
DONE
[202.46469 184.11313]
DONE
[216.54037 207.72989]


KeyboardInterrupt: 

In [63]:
env.close()

# Rename output nodes to not purely numeric names!

In [64]:
onnx_model = onnx.load('ppo_acc_bigger2_200000_steps.onnx')

In [65]:
onnx_model.graph.output[0].name = "out1"

In [66]:
onnx_model.graph.node[len(onnx_model.graph.node)-1].output[0]="out1"

In [67]:
onnx_model.graph.input[0].name

'input.1'

In [68]:
onnx.save(onnx_model, 'ppo_acc_bigger2_200000_steps.onnx')